# HW06: ML and Causal Inference (due November 9th)

**As a first part of THIS WEEK'S homework (HW06) you will have to provide feedback to two of your classmates' essays on Eduflow.** On Eduflow, you will be automatically assigned to the two essays you have to provide feedback to on Friday, in case you want to start ahead.

### For the coding part of the homework, you will have to solve only one of the three exercises.

## Exercise 1: Double Machine learning with XGboost

In this exercise you will investigate a research question similar to the one in HW02. Namely, what is the effect of being in a Union on wages? You will use the same data as in HW02 (description of the variables can be found [here](https://rdrr.io/rforge/sampleSelection/man/nlswork.html)). Although, here instead of just including controls, you will estimate the effect of union membership on wages using double machine learning and you will use XGboost as machine learning algorithm. The regression of reference is the following: 

<center> $ln\_wage_i = \beta_0 + \beta_1 union_i +\varepsilon_i$ </center>

In [1]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r16/nlswork.dta')
df = df.dropna()

In [2]:
df.head()

,idcode,year,birth_yr,age,race,msp,nev_mar,grade,collgrad,not_smsa,...,south,ind_code,occ_code,union,wks_ue,ttl_exp,tenure,hours,wks_work,ln_wage
2,1,72,51,20.0,black,1.0,0.0,12.0,0,0.0,...,0.0,4.0,6.0,1.0,0.0,2.256410,0.916667,40.0,51.0,1.589977
5,1,77,51,25.0,black,0.0,0.0,12.0,0,0.0,...,0.0,12.0,8.0,0.0,0.0,3.775641,1.500000,32.0,52.0,1.778681
7,1,80,51,28.0,black,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,5.294872,1.833333,45.0,75.0,2.551715
9,1,85,51,33.0,black,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,7.160256,1.916667,42.0,97.0,2.614172
10,1,87,51,35.0,black,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,8.987180,3.916667,45.0,95.0,2.536374


In [3]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [4]:
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

## Run the regression written above
olsf = smf.ols("ln_wage ~ union", df).fit(cov_type='hc1')
print(summary_col(olsf))


               ln_wage 
-----------------------
Intercept      1.6564  
               (0.0045)
union          0.2502  
               (0.0089)
R-squared      0.0523  
R-squared Adj. 0.0522  
Standard errors in
parentheses.


**What could be potential sources of bias (i.e. confounders)?**

In [26]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'wks_work', 'ind_code', 'grade', 'collgrad' ]]

In [27]:
# from sklearn.ensemble import RandomForestClassifier 
# from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, mean_squared_error)

# rfc_a = RandomForestClassifier(n_estimators=100, max_depth =30, random_state=42)
# rfc_b = RandomForestClassifier(n_estimators=100,  max_depth =30, random_state=42)

# rfr_a = RandomForestRegressor(n_estimators=100, max_depth =30, random_state=42)
# rfr_b = RandomForestRegressor(n_estimators=100,  max_depth =30, random_state=42)

# split into sample A and sample B
X_a, X_b, D_a, D_b, y_a, y_b= train_test_split(X, D, Y, test_size=0.5)


# Within each sample, make a validation set for xgboost early stopping
#TODO
X_atr, X_ats, D_atr, D_ats, y_atr, y_ats, = train_test_split(X_a, D_a, y_a, test_size=0.2)
X_btr, X_bts, D_btr, D_bts, y_btr, y_bts = train_test_split(X_b, D_b, y_b, test_size=0.2)

In [28]:
# Step 1. In both samples, train an xgboost regressor model to predict log wages (outcome Y)
# use early stopping.
from xgboost import XGBRegressor
xgbr_a = XGBRegressor()
xgbr_b = XGBRegressor()

xgbr_a.fit(X_atr, y_atr)
y_hat_a = xgbr_a.predict(X_ats)
xgbr_b.fit(X_btr, y_btr)
y_hat_b = xgbr_b.predict(X_bts)


print('RMSE on Test Set')
print('Sample A: %f' % np.sqrt(mean_squared_error(y_hat_a, y_ats)))
print('Sample B: %f' % np.sqrt(mean_squared_error(y_hat_b, y_bts)))

[13:35:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:35:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE on Test Set
Sample A: 0.382366
Sample B: 0.393297


In [29]:
# Step 2. In both samples, train an xgboost classifier model to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier
xgbc_a = XGBClassifier()
xgbc_b = XGBClassifier()

xgbc_a.fit(X_atr, D_atr)
D_hat_a = xgbc_a.predict(X_ats)
xgbc_b.fit(X_btr, D_btr)
D_hat_b = xgbc_b.predict(X_bts)


print('F1 Score on Test Set')
print('Sample A: %f' % f1_score(D_hat_a, D_ats))
print('Sample B: %f' % f1_score(D_hat_b, D_bts))

F1 Score on Test Set
Sample A: 0.086667
Sample B: 0.062893


In [31]:
# Step 3. Cross-fitting: Form predictions in other sample.

# predict wages in sample A using model trained in sample B:
#TODO
y_hat_a = xgbr_b.predict(X_a)

# vice versa:
#TODO
y_hat_b = xgbr_a.predict(X_b)
    
# predict union status in sample A using model trained in sample B:
#TODO
D_hat_a = xgbc_b.predict(X_a)
    
# vice versa:
#TODO
D_hat_b = xgbc_a.predict(X_b)

Recall that residuals for a variable $Z_i$ are computed as follows:

<center>$\tilde{Z}_i = Z_i - \hat{Z}_i$</center>

where $\hat{Z}_i$ is the predicted value of $Z_i$

In [50]:
D_tilde_B

11724    0.0
6821     0.0
9977     0.0
13746    0.0
12624    1.0
        ... 
6462     1.0
24283    0.0
27774    0.0
14256    0.0
4989     1.0
Name: union, Length: 6726, dtype: float64

In [52]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
y_tilde_A =  y_a - y_hat_a
y_tilde_B =  y_b - y_hat_b

# residualized union status in samples A and B:
D_tilde_A =  D_a - D_hat_a #TODO
D_tilde_B =  D_b - D_hat_b #TODO

df_dml = pd.DataFrame()

df_dml['wage_a'] = pd.Series(y_tilde_A)
df_dml['wage_b'] = pd.Series(y_tilde_B)
df_dml['union_a'] = pd.Series(D_tilde_A)
df_dml['union_b'] = pd.Series(D_tilde_B, dtype='float64')

df_dml.head()

,wage_a,wage_b,union_a,union_b
4061,-0.091771,NaN,0.0,NaN
23950,0.227085,NaN,0.0,NaN
17076,-0.023480,NaN,1.0,NaN
2652,0.293253,NaN,1.0,NaN
25149,0.187642,NaN,0.0,NaN


In [57]:
# Step 4. Run OLS regressions of the residualized variables and produce Double ML estimate
## use robust standard errors 
df_dml = pd.DataFrame({'x':D_tilde_A, 'y':y_tilde_A})
res_A = smf.ols('y~x', data=df_dml).fit()
beta_A = res_A.params
se_A = res_A.bse

df_dml = pd.DataFrame({'x':D_tilde_B, 'y':y_tilde_B})
res_B = smf.ols('y~x', data=df_dml).fit()
beta_B = res_B.params
se_B = res_B.bse

##Take the average of the two coefficient and standard errors and show the result
beta_avg = (beta_A + beta_B)/2
se_avg = (se_A + se_B)/2
print(f'beta: {beta_avg}')
print(f'se: {se_avg}')

beta: Intercept   -0.045910
x            0.196016
dtype: float64
se: Intercept    0.005240
x            0.010902
dtype: float64


**Compare the estimates form this and the previous regression. How do these change?**

The standard error is much lower with this regression and the weights of the parameters are also lower. With double ML, we ensure that our model is robust and it reduces the bias.

**Interpret the coefficient from the double ml approach. Can it be interpreted as causal?**

Yes, since the standard error is much lower, we can interpret a causality between the wage and the union variables.

## Exercise 2: Synthetic Control Method

In this exercise we will investigate the economic effects of conflict by focusing on the Basque Country. In particular, we will focus on terrorist conflicts in late 1960's in this area and use as comparison group all other regions of Spain that did not experience terrorism. The (raw) reference regression is the following:

<center>$gdpcap_r = \beta_0 + \beta_1 terrorism_r + \varepsilon_r$</center>

where $gdpcap_r$ is GDP per capita in region $r$ and $terrorism_r$ is a dummy equal to 1 if terrorist conflict happened in that region.

In [ ]:
##note that to work correctly you will also have to downgrade scipy to version 1.4.1
!pip install scipy==1.4.1
!pip install SyntheticControlMethods

#restart the kernel after downrgading scipy

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_csv('data/HW06_data_scm.csv')
del df['Unnamed: 0']
del df['regionno']
df = df.loc[df["regionname"] != "Spain (Espana)"] 

df['terrorism'] = df['regionname']=='Basque Country (Pais Vasco)'
df['terrorism'] = df['terrorism'].astype('int')
df.head()

,regionname,year,gdpcap,sec.agriculture,sec.energy,sec.industry,sec.construction,sec.services.venta,sec.services.nonventa,school.illit,school.prim,school.med,school.high,school.post.high,popdens,invest,terrorism
43,Andalucia,1955,1.688732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
44,Andalucia,1956,1.758498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
45,Andalucia,1957,1.827621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
46,Andalucia,1958,1.852756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
47,Andalucia,1959,1.878035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
import seaborn as sns

##plot gdp per capita over time separately for basque country and other regions of spain
## (i.e., separately for terrorism == 1 or 0)

**Does the parallel trend assumption hold? If not, why do you think this is the case?**

In [ ]:
## run a regression of terrorism on gdp per capita

**What could be potential threats to a causal interpretation of this result?**

In [ ]:
from SyntheticControlMethods import Synth, DiffSynth
del df['terrorism']

##fit the synthetic control 


In [ ]:
##visualize the plot 


**What can you conclude about the effect of conflict on GDP?**

In [ ]:
## Show which spanish regions were used to create the synthetic control

## Exercise 3: Heterogenous Treatment Effects

In this exercise we will investigate the effect of case management on mental health outcomes. These data come from a randomized control trial where patients were assigned to *intensive* or *standard* case management. In this context, the treatment is being assigned to the **intensive** case management while patients assigned to the **standard** case management belong to the control group..

In this exercise we will focus on heterogenous treatment effects, rather than on isolating the casual effect of the treatment. In particular, we will investigate characteristics of individuals who are most and least responsive to the treatment, i.e., to being assigned to the intensive case management. 

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta')
df = df.dropna()
pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta', iterator=True).variable_labels()

{'trialid': 'Trial ID',
 'centreid': 'Trial centre',
 'status': 'Patient status at baseline',
 'age': 'Age in years at baseline',
 'sex': 'Sex',
 'afcarib': 'Ethnic group',
 'ocfabth': "Father's social class at birth",
 'chron1l': 'Months since onset of psychosis, logged',
 'hos94': 'Days in hospital for psychiatric reasons: 2 years before baseline',
 'cprs94': 'Psychopathology at baseline (CPRS)',
 'das94': 'Disability at baseline (DAS)',
 'sat94': '(Dis)satisfaction with services at baseline',
 'rand': 'Randomised group',
 'hos96': 'Days in hospital for psychiatric reasons: 2 years after baseline',
 'cprs96': 'Psychopathology at 2 years (CPRS)',
 'sat96': '(Dis)satisfaction with services at 2 years'}

In [ ]:
df.head()

,trialid,centreid,status,age,sex,afcarib,ocfabth,chron1l,hos94,cprs94,das94,sat94,rand,hos96,cprs96,sat96
1,107.0,St George's,Out-patient,27.0,male,Other,A,3.178054,80.0,4.0,0.285714,18.0,Intensive case management,27.0,3.0,22.000
2,222005.0,St Mary's,In hospital,41.0,male,Other,D,4.521789,240.0,6.0,0.750000,15.0,Intensive case management,15.0,13.0,9.000
3,222018.0,St Mary's,In hospital,25.0,male,Other,C2,4.094345,48.0,12.0,0.125000,18.0,Intensive case management,263.0,6.0,21.375
5,312015.0,King's,Out-patient,31.0,female,Other,A,4.787492,60.0,28.0,2.375000,20.0,Intensive case management,45.0,19.0,17.000
6,221023.0,St Mary's,In hospital,35.0,male,Afro-Caribbean,C2,4.430817,60.0,25.0,1.571428,24.0,Intensive case management,58.0,27.0,19.125


The treatment variable is $rand$, the post-treatment outcomes are $hos96$, $cprs96$ and $sat96$.

In [ ]:
treatvar = 'rand'
df[treatvar].value_counts()

Intensive case management    130
Standard case management     116
Name: rand, dtype: int64

In [ ]:
outcomes = ['sat96', 'hos96', 'cprs96']
df[outcomes].describe()

,sat96,hos96,cprs96
count,246.000000,246.000000,246.000000
mean,17.271341,65.500000,17.790587
std,4.723009,104.046722,14.090911
min,9.000000,0.000000,0.000000
25%,14.000000,0.000000,7.000000
50%,17.000000,15.000000,15.000000
75%,20.187500,93.500000,26.000000
max,32.000000,692.000000,71.000000


In addition to these variables we need a set of covariates that we want to use to identify individuals who are most and least responsive to treatment. We also encode categorical covariates and prepare them for the ML model

In [ ]:
#Encoding Categorical covariates and preparing the data for tensorflow
covariates = ['status', 'sex', 'sat94', 'ocfabth', 'hos94', 'das94', 'cprs94', 'age', 'afcarib']
covariates_cat = ['status', 'sex', 'ocfabth', 'afcarib']

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[covariates_cat] = encoder.fit_transform(df[covariates_cat])
df[covariates] = df[covariates].astype('float32')
df[covariates].head()

,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib
1,1.0,1.0,18.0,0.0,80.0,0.285714,4.0,27.0,1.0
2,0.0,1.0,15.0,4.0,240.0,0.750000,6.0,41.0,1.0
3,0.0,1.0,18.0,3.0,48.0,0.125000,12.0,25.0,1.0
5,1.0,0.0,20.0,0.0,60.0,2.375000,28.0,31.0,1.0
6,0.0,1.0,24.0,3.0,60.0,1.571428,25.0,35.0,0.0


In [ ]:
# Subset the dataset by treatment and control
# Within each sample, create a training, a test and a validation set

df_treat = #TODO
df_control = #TODO

In [ ]:
##choose one of the three outcomes to analyze
## using xgboost regressor train two model to predict the outcome from the covariates
## the first model should be trained on the treated sample, while the second on the control
## use early stopping

from xgboost import XGBRegressor
xgbr_treat = XGBRegressor()
xgbr_control = XGBRegressor()

In [ ]:
##get predicted outcomes using the combined test sets for both models 

test_set = Xtreat_test.append(Xcontrol_test) 
y_hat_treat = ...
y_hat_control = ...

In [ ]:
##find the three individuals in the test set that are most and least responsive to the treatment
##namely the three individuals for who the treatment effect is larger and those for who it is smaller

test_set['treat_effect'] = ...

In [ ]:
##visualize and comment on the covariates of these individuals 